In [1]:
# Import Libraries
from unsloth import FastLanguageModel
from datasets import Dataset
import pandas as pd
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bf16_supported
import os

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/student/anaconda3/envs/llama/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
# Load local model
path = "Llama-3.2-1B-Instruct-Uncensored"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=path)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.0.
   \\   /|    NVIDIA RTX 4000 Ada Generation. Num GPUs = 1. Max memory: 19.526 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
# Load the Dataset
"""NEED TO PREPROCESS THE DATA BEFORE TRAINING"""
dataset_path = "datasets/annotated_tweets_w_text.csv"
df = pd.read_csv(dataset_path)

# Pandas to Dataset
dataset = Dataset.from_pandas(df)

print(dataset)

EOS_TOKEN = tokenizer.eos_token
def format_tweets(data): 
    tweets = data['Text']
    texts = []
    for tweet in tweets: 
        text = tweet + EOS_TOKEN
        texts.append(text)

    return { "text": texts, }

formatted_dataset = dataset.map(format_tweets, batched=True)

Dataset({
    features: ['Tweet ID', 'Text', 'label'],
    num_rows: 2290
})


Map: 100%|██████████| 2290/2290 [00:00<00:00, 489499.35 examples/s]


In [4]:
# Show the formatted dataset
formatted_dataset['text'][0]

'Are we still allowed to quote ancient Chinese proverbs, or is that racist? #RacismIsAVirus<|eot_id|>'

In [5]:
# Set up the training
model = FastLanguageModel.get_peft_model(
    model, 
    r=16, 
    target_modules= [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ], 
    lora_alpha=16,
    lora_dropout=0.05, 
    bias="none", 
    use_gradient_checkpointing="unsloth", 
    random_state=3407,
    use_rslora=True, 
    loftq_config=None, )

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.3.19 patched 16 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
max_seq_length = 2048

trainer = SFTTrainer(
    model = model, 
    tokenizer = tokenizer, 
    train_dataset = formatted_dataset, 
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 6,
    args = TrainingArguments(
        per_device_train_batch_size = 128,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 10,
        learning_rate = 2e-4,
        fp16 = not is_bf16_supported(),
        bf16 = is_bf16_supported(),
        logging_steps = 50,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "checkpoints",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6): 100%|██████████| 2290/2290 [00:01<00:00, 2218.84 examples/s]


In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,290 | Num Epochs = 3 | Total steps = 10
O^O/ \_/ \    Batch size per device = 128 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (128 x 4 x 1) = 512
 "-____-"     Trainable parameters = 11,272,192/1,000,000,000 (1.13% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss


In [ ]:
# Save merged model and save gguf
model.save_pretrained_gguf("output", tokenizer, quantization_method = "q8_0")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 78.95 out of 125.5 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 16/16 [00:00<00:00, 64.44it/s]


Unsloth: Saving tokenizer... Done.
Done.
